In [29]:
import pandas as pd
from pathlib import Path
import numpy as np

base_dir = Path(r"I:\NTS\classified builds")

In [2]:
v10 = pd.read_csv(base_dir / "cb_tfn_v10.csv")

C:\Users\Predator\AppData\Local\Temp\ipykernel_6636\4015051739.py:1: DtypeWarning: Columns (9,59) have mixed types. Specify dtype option on import or set low_memory=False.
  v10 = pd.read_csv(base_dir / "cb_tfn_v10.csv")


In [4]:
v10.columns

Index(['PSUID', 'PSUPSect', 'PSUPopDensity', 'PSUAreaType1_B01ID',
       'PSUAreaType2_B01ID', 'PSUGOR_B02ID', 'PSUCounty_B01ID', 'SurveyYear',
       'HouseholdID', 'HHoldOSWard_B01ID', 'HHoldOSLAUA_B01ID',
       'HHoldCounty_B01ID', 'HHoldGOR_B02ID', 'HHoldAreaType1_B01ID',
       'HHoldAreaType2_B01ID', 'HHoldNumAdults', 'NumCarVan_B02ID',
       'NumCarVan', 'W1', 'W2', 'W3', 'IndividualID', 'Age_B01ID', 'Sex_B01ID',
       'XSOC2000_B02ID', 'NSSec_B03ID', 'SIC2007_B02ID', 'CarAccess_B01ID',
       'DrivLic_B02ID', 'EcoStat_B01ID', 'EcoStat_B02ID', 'DayID', 'TravDay',
       'TravelWeekDay_B01ID', 'TravelWeekDay_B02ID', 'TripID', 'JJXSC',
       'JOTXSC', 'JTTXSC', 'JD', 'MainMode_B03ID', 'MainMode_B04ID',
       'MainMode_B11ID', 'TripPurpFrom_B01ID', 'TripPurpTo_B01ID',
       'TripStart_B01ID', 'TripEnd_B01ID', 'TripDisIncSW',
       'TripDisIncSW_B01ID', 'TripTravTime', 'TripOrigCounty_B01ID',
       'TripDestCounty_B01ID', 'TripDestUA2009_B01ID', 'TripOrigUA2009_B01ID',
    

In [64]:
edges = [0, 10, 25, 50, 100, 200, 1000]
labels = ['0-10', '10-25','25-50','50-100','100-200','200+']
v10['dist_bins'] = pd.cut(v10.TripDisIncSW, bins=edges, labels=labels)

In [65]:
modes = {1: 'Walk',
  2: 'Cycle',
  3: 'Car Driver',
  4: 'Car Passenger',
  5: 'Bus',
  6: 'rail',
  8: 'Air'}

In [35]:
purposes = {1: 'Commute',
  2: 'Business',
  3: 'Education',
  4: 'Shopping',
  5: 'PB',
  6: 'Social',
  7: 'Visit Friends',
  8: 'Holiday - Day trip',
  11: 'Escort commute',
  12: 'Escort business',
  13: 'Escort education',
  14: 'Escort shopping'}

In [38]:
area_types = {1: 'Inner London',
  2: 'Outer London',
  3: 'Metropolitan built-up',
  4: 'Urban Big ( > 250k population)',
  5: 'Urban Large (100k to 250k population)',
  6: 'Urban Medium (25k to 100k population)',
  7: 'Small Urban ( < 25k)',
  8: 'Rural'}

In [39]:
other_dic = {0.0: 'other',
 'Shopping': 'other',
 'Social': 'other',
 'Visit Friends': 'other',
 'Holiday - Day trip': 'other',
 'Escort commute': 'other',
 'Escort business': 'other',
 'Escort shopping': 'other'}

In [40]:
v10 = v10.set_index('tfn_at').rename(index=area_types).reset_index()
v10 = v10.set_index('trip_purpose').rename(index=purposes).reset_index()
v10 = v10.set_index('main_mode').rename(index=modes).reset_index()


In [42]:
v10 = v10.set_index('trip_purpose').rename(index=other_dic).reset_index()

In [66]:
table_1 = v10.groupby(['tfn_at','trip_purpose','dist_bins','main_mode']).sum()['weighted_trips']

C:\Users\Predator\AppData\Local\Temp\ipykernel_6636\3153417575.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_1 = v10.groupby(['tfn_at','trip_purpose','dist_bins','main_mode']).sum()['weighted_trips']


In [67]:
table_2 =v10[v10['main_mode'].str.contains('Car', na=False)].groupby(['tfn_at','trip_purpose','dist_bins','occupant']).sum()['weighted_trips'].unstack()

C:\Users\Predator\AppData\Local\Temp\ipykernel_6636\3363942408.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_2 =v10[v10['main_mode'].str.contains('Car', na=False)].groupby(['tfn_at','trip_purpose','dist_bins','occupant']).sum()['weighted_trips'].unstack()


In [71]:
table_2['occupants'] = (table_2['driver'] + table_2['passenger']) / table_2['driver']

In [73]:
table_2.to_csv(r"I:\NTS\outputs\city_science_tables\table_2_v2.csv")

In [74]:
table_1.to_csv(r"I:\NTS\outputs\city_science_tables\table_1_v2.csv")